In [1]:
import time
import asyncio
import aiohttp
import requests
import threading
import pandas as pd

In [2]:
cols = ['id', 'name',  'age_ratings', 'slug', 'aggregated_rating', 'aggregated_rating_count', 'category', 'collection', 
        'alternative_names', 'external_games', 'first_release_date', 'game_engines', 'game_modes', 'genres', 
        'involved_companies', 'keywords', 'platforms', 'rating', 'player_perspectives', 'dlcs', 'franchise',
        'rating_count', 'release_dates', 'similar_games', 'tags', 'parent_game', 'status',
        'total_rating', 'total_rating_count', 'updated_at', 'websites']

In [3]:
client_id = 'x0rncah74zueqqov31fhkwuzlqv27a'
access_token = 'Bearer lhty0tvjq3el3s4rufgiw5vufmjk31'

In [4]:
async def create_games_dataset(floor, ceiling, endpoint='games'):
    # нужно для того, чтобы авторизироваться
    headers = {'Client-ID': client_id, 
               'Authorization': access_token}
    async with aiohttp.ClientSession() as session:
        all_games = pd.DataFrame(columns=cols)
        for i in range(floor, ceiling):
            # endpoint и id можно изменять для того, чтобы получить разные наборы
            body = 'fields *; where id = ' + str(i) + ';'
            async with session.post('https://api.igdb.com/v4/' + endpoint + '/', headers=headers, data=body) as resp:
                game = await resp.json()
                try: # если вышла такая ошибка, значит слишком много запросов в одну секунду
                    # на тот случай, если id нет в базе данных
                    if len(game) > 0:
                        all_games.loc[len(all_games.index)] = game[0]
                except KeyError: # поэтому, ждем 1 секунду и делаем новый запрос
                    time.sleep(3)
                    async with session.post('https://api.igdb.com/v4/' + endpoint + '/', headers=headers, data=body) as resp:
                        game = await resp.json()
                        if len(game) > 0:
                            all_games.loc[len(all_games.index)] = game[0]
        return all_games

In [5]:
start_time = time.time()
all_fields_one = await create_games_dataset(1, 40000)
print("--- %s seconds ---" % (time.time() - start_time))

--- 29412.863607645035 seconds ---


In [7]:
all_fields_one.to_csv('C:/Users/ASDW/Python/Projects/Recosys 2.0/datasets/games_dataset/new_games_dataset.csv', mode = 'a', index=False)

In [6]:
all_fields_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37812 entries, 0 to 37811
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       37811 non-null  object 
 1   name                     37811 non-null  object 
 2   age_ratings              12746 non-null  object 
 3   slug                     37811 non-null  object 
 4   aggregated_rating        11039 non-null  float64
 5   aggregated_rating_count  11039 non-null  object 
 6   category                 37811 non-null  object 
 7   collection               14003 non-null  object 
 8   alternative_names        10780 non-null  object 
 9   external_games           37096 non-null  object 
 10  first_release_date       34906 non-null  object 
 11  game_engines             4157 non-null   object 
 12  game_modes               31641 non-null  object 
 13  genres                   34841 non-null  object 
 14  involved_companies    